## 1. Activation functions.
***

!['activation'](activation_function.png)

In [1]:
import torch
import torch.nn as nn

In [2]:
#The neural net blueprint is implemented with pytorch below.
input_layer = torch.tensor([2., 1.])
weight_1 = torch.tensor([[0.45, 0.32], [-0.12, 0.29]])

hidden_layer = torch.matmul(input_layer, weight_1)

weight_2 = torch.tensor([[0.48, -0.12], [0.64, 0.91]])
output_layer = torch.matmul(hidden_layer, weight_2)
print(output_layer)

tensor([0.9696, 0.7527])


In [3]:
weight = torch.matmul(weight_1, weight_2)
print(weight)

tensor([[0.4208, 0.2372],
        [0.1280, 0.2783]])



#### To compute a layer, you need to multiply the previous layer by the weights of connections between the layers. 
***

### 1.1 ReLU activation function.

In [4]:
relu = nn.ReLU()

tensor_1 = torch.tensor([2., -4.])
print(relu(tensor_1))

tensor_2 = torch.tensor([[2., -4.], 
                        [1.2, 0.]])
print(relu(tensor_2))

tensor([2., 0.])
tensor([[2.0000, 0.0000],
        [1.2000, 0.0000]])


In [5]:
# Instantiate ReLU activation function as relu
relu = nn.ReLU()

input_layer = torch.rand(4)
# Initialize weight_1 and weight_2 with random numbers
weight_1 = torch.rand(4, 6)
weight_2 = torch.rand(6, 2)

# Multiply input_layer with weight_1
hidden_1 = torch.matmul(input_layer, weight_1)

# Apply ReLU activation function over hidden_1 and multiply with weight_2
hidden_1_activated = relu(hidden_1)
print(torch.matmul(hidden_1_activated, weight_2))

tensor([6.1240, 4.1571])


## 2. Loss functions.
***

In [6]:
#Initialize the scores and ground truth
logits = torch.tensor([[-1.2, 0.12, 4.8]])
ground_truth = torch.tensor([2])

#Instantiate the cross-entropy loss.
criterion = nn.CrossEntropyLoss()

#Compute and print the loss.
loss = criterion(logits, ground_truth)
print(loss)

tensor(0.0117)


## 3. Preparing a dataset in Pytorch.

#### We'll use `MNIST` and `CIFAR-10` datasets.
***

In [7]:
import torchvision
import torch.utils.data
import torchvision.transforms as transforms

In [8]:
#Define the transformation of images to torch tensors.
transform = transforms.Compose([
    transforms.ToTensor(),
    #Standardize the images in each channel. (Mean, Std.Dev)
    transforms.Normalize((0.4914, 0.48216, 0.44653),
                         (0.24703, 0.24349, 0.26159))
])

In [9]:
#Obtain the dataset.
trainset = torchvision.datasets.CIFAR10(root = './data', train = True, 
                                       download = True, transform = transform)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, 
                                      download = True, transform = transform)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
#Build trainloader and testloader to get the dataset ready for Pytorch.
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 32, 
                                         shuffle = True, num_workers = 4)

testloader = torch.utils.data.DataLoader(testset, batch_size = 32, 
                                        shuffle = False, num_workers = 4)


In [11]:
#Inspect the batch size.
print(testloader.batch_size)

#Inspact the type of random sampler.
print(trainloader.sampler)

32


## 4. Training neural nets.
***

#### The procedure for training a neural net looping over the dataset while:
- Do a forward pass.
- Calculate the loss function.
- Calculate the gradients.
- Change the weights based on the gradients.

In [12]:
import torch.nn.functional as F
import torch.optim as optim

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 500) #Input shape same as the CIFAR-10 dataset shape.
        self.fc2 = nn.Linear(500, 10) #500 input nodes and 10 output nodes.
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [ ]:
#Instantiate the network, the loss function and the optimizer.
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 3e-4)

for epoch in range(10): #Loop over the dataset multiple times.
    for i, data in enumerate(trainloader, 0):
        #Get the inputs.
        inputs, labels = data
        inputs = inputs.view(-1, 32 * 32 * 3) #Put images into vectors
        
        #Zero the parameter gradients.
        optimizer.zero_grad()
        
        #Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [ ]:
#Perform inference.
correct, total = 0, 0
predictions = []
net.eval() #Evaluation mode.

for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs = inputs.view(-1, 32 * 32 * 3)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    predictions.append(outputs)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
    
print('The test set accuracy is: %d %%' % (100 * correct / total))